In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
import glob as glob
import os
from IPython.display import SVG
#from math import nan, isnan  # I think I don't need it anymore
from itertools import combinations # added this one too.

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")
    

saved_csv folder already exists.
Music_Files folder already exists.


In [10]:
import textdistance as td

ALGORITHMS = [
    "Levenshtein",
    "Hamming",
]

CALCULATIONS = ["Distance", "Similarity"]


algorithm_to_apply = {
        ("Levenshtein", "Distance", True): td.levenshtein.normalized_distance,
        ("Levenshtein", "Similarity", True): td.levenshtein.normalized_similarity,
        ("Levenshtein", "Distance", False): td.levenshtein.distance,
        ("Levenshtein", "Similarity", False): td.levenshtein.similarity,
        
        ("Hamming", "Distance", True): td.hamming.normalized_distance,
        ("Hamming", "Similarity", True): td.hamming.normalized_similarity,
        ("Hamming", "Distance", False): td.hamming.distance,
        ("Hamming", "Similarity", False): td.hamming.similarity,
        
    }

In [16]:
#import one of your pieces for voice number detector

#4vv
piece = importScore('https://crimproject.org/mei/CRIM_Mass_0033_1.mei')

#5vv
#piece = importScore('https://crimproject.org/mei/CRIM_Mass_0020_1.mei')
#6vv
#piece = importScore('https://crimproject.org/mei/CRIM_Mass_0015_1.mei')

voice_number = len(piece.notes().columns)
print(piece.metadata['composer'],"'s", piece.metadata['title'], 'has', voice_number, 'voices.')


Previously imported piece detected.
Roland de Lassus 's Missa Doulce memoire: Kyrie has 4 voices.


In [17]:
#piece = importScore('Music_Files/CRIM_Model_0033.mei')
#piece = importScore('https://crimproject.org/mei/CRIM_Model_0003.mei')
#corpus = CorpusBase(['Music_Files/CRIM_Model_0033.mei'])
#corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0001.mei'])

#4vv
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0033.mei', 
                     'https://crimproject.org/mei/CRIM_Mass_0033_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0033_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0033_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0033_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0033_5.mei'])  

"""
#5vv
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0020_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0020_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0020_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0020_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0020_5.mei'])   
                     
#6vv
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0015_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_5.mei'])                  
              
"""

Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Error downloading https://crimproject.org/mei/CRIM_Mass_0033_4.mei, please check your url and try again. Continuing to next file.
Previously imported piece detected.


"\n#5vv\ncorpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0020_1.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0020_2.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0020_3.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0020_4.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0020_5.mei'])   \n                     \n#6vv\ncorpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0015_1.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0015_2.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0015_3.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0015_4.mei',\n                     'https://crimproject.org/mei/CRIM_Mass_0015_5.mei'])                  \n              \n"

In [18]:
pd.options.display.max_colwidth = 200
#pd.set_option('display.max_rows', 5000)

In [19]:
quote_length = 10

func1 = ImportedPiece.harmonic
list_of_dfs = corpus.batch(func=func1, kwargs={'kind': 'd', 'compound':False, 'directed':False}, metadata=False) # I added 'directed':False to avoid negatives and be able to have integers but this can create false exact matches 
func2 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func2, kwargs={'offset':True,'df': list_of_dfs})

cleaned_list = []

for harm in list_of_detail_index: #for each df sitting in the list applying these functions (for each df in list, can apply df functions, but not on list as a whole)
    harm = harm.fillna(method="pad")
    harm = harm.replace('Rest', '0')
    harm['figures'] = harm[harm.columns[:voice_number-1]].apply(lambda x: ''.join(x.astype(str)), axis=1) #a default 'figures' value if lowest voice is sounding
        #harm['figures'] = harm[harm.columns[:3]].apply(lambda x: ''.join(x.astype(str)), axis=1)
        #harm['figures'] = harm[harm.columns[0]].astype(str) + harm[harm.columns[1]].astype(str) + harm[harm.columns[2]].astype(str)
    
    if voice_number == 4:
        #v4_rests = (harm.iloc[:, 0:3] == '0') THIS DOESNT WORK
        v4_rests = (harm.iloc[:,0] == '0') & (harm.iloc[:,1] == '0') & (harm.iloc[:,2] == '0') # .iloc[:,0] == 'Rest'  means taking all rows (:) in column zero (,0) then filtering (==Rest) for where the row is Rest
        harm.loc[v4_rests, 'figures'] = harm.loc[v4_rests, harm.columns[0]].astype(str) + harm.loc[v4_rests, harm.columns[3]].astype(str) + harm.loc[v4_rests, harm.columns[4]].astype(str)
        v3_rests = v4_rests & (harm.iloc[:,3] == '0') & (harm.iloc[:,4] == '0')
        harm.loc[v3_rests, 'figures'] = harm.loc[v3_rests, harm.columns[0]].astype(str) + harm.loc[v3_rests, harm.columns[3]].astype(str) + harm.loc[v3_rests, harm.columns[5]].astype(str)
    
    elif voice_number == 5:
        v5_rests = (harm.iloc[:,0] == '0') & (harm.iloc[:,1] == '0') & (harm.iloc[:,2] == '0') & (harm.iloc[:,3] == '0') 
        harm.loc[v5_rests, 'figures'] = harm.loc[v5_rests, harm.columns[0]].astype(str) + harm.loc[v5_rests, harm.columns[4]].astype(str) + harm.loc[v5_rests, harm.columns[5]].astype(str) + harm.loc[v5_rests, harm.columns[6]].astype(str)
        v4_rests =  v5_rests & (harm.iloc[:,4] == '0') & (harm.iloc[:,5] == '0') & (harm.iloc[:,6] == '0')
        harm.loc[v4_rests, 'figures'] = harm.loc[v4_rests, harm.columns[0]].astype(str) + harm.loc[v4_rests, harm.columns[4]].astype(str) + harm.loc[v4_rests, harm.columns[7]].astype(str)  + harm.loc[v4_rests, harm.columns[8]].astype(str)
        v3_rests =  v4_rests & (harm.iloc[:,7] == '0') & (harm.iloc[:,8] == '0')
        harm.loc[v3_rests, 'figures'] = harm.loc[v3_rests, harm.columns[0]].astype(str) + harm.loc[v3_rests, harm.columns[4]].astype(str) + harm.loc[v3_rests, harm.columns[7]].astype(str)  + harm.loc[v3_rests, harm.columns[9]].astype(str)
    
    elif voice_number == 6:
        v6_rests = (harm.iloc[:,0] == '0') & (harm.iloc[:,1] == '0') & (harm.iloc[:,2] == '0') & (harm.iloc[:,3] == '0') & (harm.iloc[:,4] == '0') 
        harm.loc[v6_rests, 'figures'] = harm.loc[v6_rests, harm.columns[0]].astype(str) + harm.loc[v6_rests, harm.columns[5]].astype(str) + harm.loc[v6_rests, harm.columns[6]].astype(str) + harm.loc[v6_rests, harm.columns[7]].astype(str) + harm.loc[v6_rests, harm.columns[8]].astype(str)
        v5_rests = v6_rests & (harm.iloc[:,5] == '0') & (harm.iloc[:,6] == '0') & (harm.iloc[:,7] == '0') & (harm.iloc[:,8] == '0') 
        harm.loc[v5_rests, 'figures'] = harm.loc[v5_rests, harm.columns[0]].astype(str) + harm.loc[v5_rests, harm.columns[5]].astype(str) + harm.loc[v5_rests, harm.columns[9]].astype(str) + harm.loc[v5_rests, harm.columns[10]].astype(str) + harm.loc[v5_rests, harm.columns[11]].astype(str)
        v4_rests =  v5_rests & (harm.iloc[:,9] == '0') & (harm.iloc[:,10] == '0') & (harm.iloc[:,11] == '0')
        harm.loc[v4_rests, 'figures'] = harm.loc[v4_rests, harm.columns[0]].astype(str) + harm.loc[v4_rests, harm.columns[5]].astype(str) + harm.loc[v4_rests, harm.columns[9]].astype(str) + harm.loc[v4_rests, harm.columns[12]].astype(str) + harm.loc[v4_rests, harm.columns[13]].astype(str)
        v3_rests =  v4_rests & (harm.iloc[:,12] == '0') & (harm.iloc[:,13] == '0')
        harm.loc[v3_rests, 'figures'] = harm.loc[v3_rests, harm.columns[0]].astype(str) + harm.loc[v3_rests, harm.columns[5]].astype(str) + harm.loc[v3_rests, harm.columns[9]].astype(str) + harm.loc[v3_rests, harm.columns[12]].astype(str) + harm.loc[v3_rests, harm.columns[14]].astype(str)
    
    
    harm = harm.loc[harm['figures'].shift() != harm['figures']] # getting rid of consecutive duplicate rows
    for i in range(1, quote_length): #creating as many new 'figures' columns as quote length requires
        harm[f'figures{i}'] = harm['figures'].shift(-i) #or "figures"+str(i)
    harm['figures_combined'] = harm[harm.columns[-quote_length:]].iloc[0:].values.tolist() #combining all 'figures{i}' as a list
   
    harm = harm.dropna().drop(harm.columns[-quote_length:-1], axis=1) #drop a row if contains NaN values, then drop all 'figures{i}' columns
    cleaned_list.append(harm)
    
harm_corpus = pd.concat(cleaned_list)
harm_corpus.head(10)

4_3 4_2 4_1 3_2 3_1 2_1        Composer           Title  \
Measure Beat Offset                                                           
1       1.0  0.0      5   8   3   4   6   3  Pierre Sandrin  Doulce memoire   
2       1.0  8.0      3   5   8   3   6   4  Pierre Sandrin  Doulce memoire   
        2.0  10.0     5   8   3   4   6   3  Pierre Sandrin  Doulce memoire   
        3.0  12.0     3   5   8   3   6   4  Pierre Sandrin  Doulce memoire   
3       1.0  16.0     5   8   3   4   6   3  Pierre Sandrin  Doulce memoire   
        3.0  20.0     0   0   0   5   0   0  Pierre Sandrin  Doulce memoire   
        4.0  22.0     0   0   0   5   8   4  Pierre Sandrin  Doulce memoire   
4       1.0  24.0     8   0   3   0   3   0  Pierre Sandrin  Doulce memoire   
        2.5  27.0     8   0   4   0   4   0  Pierre Sandrin  Doulce memoire   
        3.0  28.0     6   3   3   4   5   8  Pierre Sandrin  Doulce memoire   

                    figures  \
Measure Beat Offset           
1       1.0  0.0        583   
2       1.0  8.0        358   
        2.0  10.0       583   
        3.0  12.0       358   
3       1.0  16.0       583   
        3.0  20.0       050   
        4.0  22.0       058   
4       1.0  24.0       803   
        2.5  27.0       804   
        3.0  28.0       633   

                                                       figures_combined  
Measure Beat Offset                                                      
1       1.0  0.0     [583, 358, 583, 358, 583, 050, 058, 803, 804, 633]  
2       1.0  8.0     [358, 583, 358, 583, 050, 058, 803, 804, 633, 638]  
        2.0  10.0    [583, 358, 583, 050, 058, 803, 804, 633, 638, 527]  
        3.0  12.0    [358, 583, 050, 058, 803, 804, 633, 638, 527, 335]  
3       1.0  16.0    [583, 050, 058, 803, 804, 633, 638, 527, 335, 453]  
        3.0  20.0    [050, 058, 803, 804, 633, 638, 527, 335, 453, 363]  
        4.0  22.0    [058, 803, 804, 633, 638, 527, 335, 453, 363, 584]  
4       1.0  24.0    [803, 804, 633, 638, 527, 335, 453, 363, 584, 583]  
        2.5  27.0    [804, 633, 638, 527, 335, 453, 363, 584, 583, 838]  
        3.0  28.0    [633, 638, 527, 335, 453, 363, 584, 583, 838, 583]

In [20]:
#harm_corpus.figures.value_counts().head(30) #.unique()
# count exact quotes
temp = harm_corpus[['Composer','Title','figures_combined']].astype(str) #to use .value_counts(), I need to convert entire list into a sinle string. Idk why can't use value_counts() with a list? 
temp['figures_combined'].value_counts()[temp['figures_combined'].value_counts()>1].to_frame().head(20)
#locate exact quotes or quote fragments (substrings)
#temp[temp['figures_combined'].str.contains("683', '572', '358', '583', '636', '853'")] #each in '', no []; eg.("'583', '358', '362', '383', '324', '533', '523', '368', '373', '363'")

,figures_combined
"['538', '532', '533', '534', '835', '247', '358', '584', '583', '838']",4
"['838', '833', '823', '038', '047', '583', '472', '388', '287', '386']",3
"['735', '584', '583', '838', '833', '823', '038', '047', '583', '472']",3
"['583', '835', '724', '683', '572', '358', '583', '636', '853', '385']",3
"['533', '534', '835', '247', '358', '584', '583', '838', '538', '530']",3
"['584', '583', '838', '833', '823', '038', '047', '583', '472', '388']",3
"['583', '838', '833', '823', '038', '047', '583', '472', '388', '287']",3
"['534', '835', '247', '358', '584', '583', '838', '538', '530', '538']",3
"['835', '735', '584', '583', '838', '833', '823', '038', '047', '583']",3
"['247', '358', '584', '583', '838', '538', '530', '538', '053', '853']",3


In [25]:
# Automatic similarity detector (Takes 2.5mins to load)
harm_corpus = harm_corpus.reset_index()
harm_corpus["Location"] = harm_corpus["Composer"] + "_" + harm_corpus["Title"]+ "_" + harm_corpus["Measure"].astype(str) + "_" + harm_corpus["Beat"].astype(str)

location_list = harm_corpus['Location'].tolist()
location_list_comb = list(combinations(location_list, 2)) 
comb_location = pd.DataFrame(location_list_comb, columns = ['source_location', 'match_location']) 

figures_list = harm_corpus['figures_combined'].tolist()
figures_list_comb = list(combinations(figures_list, 2)) 
comb_dist = pd.DataFrame(figures_list_comb, columns = ['source', 'match']) 
comb_dist['distance'] = comb_dist[['source', 'match']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)

comb = pd.concat([comb_dist.reset_index(drop=True),comb_location.reset_index(drop=True)], axis=1)
comb = comb[['source','match','source_location','match_location','distance']]
comb

,source,match,source_location,match_location,distance
0,"[583, 358, 583, 358, 583, 050, 058, 803, 804, 633]","[358, 583, 358, 583, 050, 058, 803, 804, 633, 638]",Pierre Sandrin_Doulce memoire_1_1.0,Pierre Sandrin_Doulce memoire_2_1.0,2
1,"[583, 358, 583, 358, 583, 050, 058, 803, 804, 633]","[583, 358, 583, 050, 058, 803, 804, 633, 638, 527]",Pierre Sandrin_Doulce memoire_1_1.0,Pierre Sandrin_Doulce memoire_2_2.0,4
2,"[583, 358, 583, 358, 583, 050, 058, 803, 804, 633]","[358, 583, 050, 058, 803, 804, 633, 638, 527, 335]",Pierre Sandrin_Doulce memoire_1_1.0,Pierre Sandrin_Doulce memoire_2_3.0,6
3,"[583, 358, 583, 358, 583, 050, 058, 803, 804, 633]","[583, 050, 058, 803, 804, 633, 638, 527, 335, 453]",Pierre Sandrin_Doulce memoire_1_1.0,Pierre Sandrin_Doulce memoire_3_1.0,8
4,"[583, 358, 583, 358, 583, 050, 058, 803, 804, 633]","[050, 058, 803, 804, 633, 638, 527, 335, 453, 363]",Pierre Sandrin_Doulce memoire_1_1.0,Pierre Sandrin_Doulce memoire_3_3.0,10
...,...,...,...,...,...
569773,"[732, 385, 284, 853, 852, 538, 537, 835, 684, 583]","[284, 853, 852, 538, 537, 835, 684, 583, 482, 583]",Roland de Lassus_Missa Doulce memoire: Agnus Dei_22_3.5,Roland de Lassus_Missa Doulce memoire: Agnus Dei_22_4.5,4
569774,"[732, 385, 284, 853, 852, 538, 537, 835, 684, 583]","[853, 852, 538, 537, 835, 684, 583, 482, 583, 158]",Roland de Lassus_Missa Doulce memoire: Agnus Dei_22_3.5,Roland de Lassus_Missa Doulce memoire: Agnus Dei_23_1.0,6
569775,"[385, 284, 853, 852, 538, 537, 835, 684, 583, 482]","[284, 853, 852, 538, 537, 835, 684, 583, 482, 583]",Roland de Lassus_Missa Doulce memoire: Agnus Dei_22_4.0,Roland de Lassus_Missa Doulce memoire: Agnus Dei_22_4.5,2
569776,"[385, 284, 853, 852, 538, 537, 835, 684, 583, 482]","[853, 852, 538, 537, 835, 684, 583, 482, 583, 158]",Roland de Lassus_Missa Doulce memoire: Agnus Dei_22_4.0,Roland de Lassus_Missa Doulce memoire: Agnus Dei_23_1.0,4


In [22]:
#the above filtered, adjust variable
comb[(comb['source_location'] == 'Pierre Sandrin_Doulce memoire_4_3.0') & (comb['distance'] < 3)]
#df
#df[df['distance'] < 4]

#comb[comb['distance'] < 3]
#comb.loc[(comb['distance'] > 0) & (comb['distance'] < 4)]

,source,match,source_location,match_location,distance
9567,"[633, 638, 527, 335, 453, 363, 584, 583, 838, 583]","[638, 527, 335, 453, 363, 584, 583, 838, 583, 358]",Pierre Sandrin_Doulce memoire_4_3.0,Pierre Sandrin_Doulce memoire_4_4.0,2
9877,"[633, 638, 527, 335, 453, 363, 584, 583, 838, 583]","[603, 638, 527, 335, 453, 363, 584, 583, 838, 135]",Pierre Sandrin_Doulce memoire_4_3.0,Roland de Lassus_Missa Doulce memoire: Kyrie_4_3.0,2


In [23]:
    

'''
fabians_df = harm_corpus[['Composer','Title','figures_combined']].astype(str)
fabians_df = fabians_df['figures_combined'].value_counts()[fabians_df['figures_combined'].value_counts()>1].to_frame()

for index, row in fabians_df.iterrows():
    el1 = [ int(index[0][i]) for i in range(3) ]
    el2 = 

############
#create a dictionary for each figure to save computing power
memos = {}
def distance_func(source, match):
    key = tuple(source, match)
    if key in memos:
        return memos[key]
    elif tuple(match, source) in memos:
        return memos[tuple(match, source)]
    else:
        memos[key] = td.levenshtein.distance(*key)
        return memos[key]
'''


"\nfabians_df = harm_corpus[['Composer','Title','figures_combined']].astype(str)\nfabians_df = fabians_df['figures_combined'].value_counts()[fabians_df['figures_combined'].value_counts()>1].to_frame()\n\nfor index, row in fabians_df.iterrows():\n    el1 = [ int(index[0][i]) for i in range(3) ]\n    el2 = \n\n############\n#create a dictionary for each figure to save computing power\nmemos = {}\ndef distance_func(source, match):\n    key = tuple(source, match)\n    if key in memos:\n        return memos[key]\n    elif tuple(match, source) in memos:\n        return memos[tuple(match, source)]\n    else:\n        memos[key] = td.levenshtein.distance(*key)\n        return memos[key]\n"

In [24]:
#similarity detector for given quote
#my_quote =  ['363', '583', '830', '583', '835', '724', '683', '572', '358', '583', '636', '853'] # THIS IS A LIST #['583', '358', '583', '358', '583', '050', '058', '803'] 

'''
my_quote = [my_quote for i in range(len(harm_corpus))]
df = harm_corpus[['Composer','Title','figures_combined']].copy() #.reset_index() #harm_corpus['figures_combined'].to_frame() #harm_corpus[['Composer','Title','figures_combined']].astype(str)
df.rename(columns={'figures_combined':'similar_passage'}, inplace=True)
df["my_quote"] = my_quote
df['distance'] = df[['similar_passage', 'my_quote']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)
df
#the above filtered, adjust variable
df[df['distance'] < 5]

'''

'\nmy_quote = [my_quote for i in range(len(harm_corpus))]\ndf = harm_corpus[[\'Composer\',\'Title\',\'figures_combined\']].copy() #.reset_index() #harm_corpus[\'figures_combined\'].to_frame() #harm_corpus[[\'Composer\',\'Title\',\'figures_combined\']].astype(str)\ndf.rename(columns={\'figures_combined\':\'similar_passage\'}, inplace=True)\ndf["my_quote"] = my_quote\ndf[\'distance\'] = df[[\'similar_passage\', \'my_quote\']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)\ndf\n#the above filtered, adjust variable\ndf[df[\'distance\'] < 5]\n\n'

In [14]:
#TRASH
#type(harm_corpus['figures_combined']) #series
#type(harm_corpus.iloc[0,9]) #list; each row in figures_combined is a list, even though each element within it isnt in ''
#type(harm_corpus.iloc[0,9][0]) #str

#my_list = ['A', 'B','C', 'D']
#my_big_list = list(combinations(my_list, 2))
#df_list = pd.DataFrame(my_big_list, columns = ['source', 'match'])
#df_list

#TESTING # This is working automatic similarity detector
#df_corpus_comb_list = pd.DataFrame(corpus_comb_list, columns = ['source', 'match']) 
#df_corpus_comb_list
    #type(df_corpus_comb_list.iloc[0,0][0])
    #type(harm_corpus.iloc[0,9]) #list
    #type(harm_corpus.iloc[0,9][0]) #str#TESTING
#df_corpus_comb_list['similarity'] = df_corpus_comb_list[['source', 'match']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)
#df_corpus_comb_list
#filtered_dist = df_corpus_comb_list.loc[(df_corpus_comb_list['similarity'] > 0) & (df_corpus_comb_list['similarity'] < 4)]
#filtered_dist


# OLD similarity detector for given quote # how to not need [] and ''?? (each column in harm_corpus is object;harm_corpus.dtypes); #terribly slow, how to make faster? 
#my_quote = "['724', '683', '572', '358', '583', '636', '853', '385']" #['583', '835', '724', '683', '572', '358', '583', '636', '853', '385', '484', '584', '583', '838']" #need [] and ''
#df = harm_corpus[['Composer','Title','figures_combined']].astype(str)
#df.rename(columns={'figures_combined':'similar_passage'}, inplace=True)
#df["my_quote"] = my_quote
#df['similarity'] = df[['similar_passage', 'my_quote']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)

#automatic similarity detector via generating lists with ngrams method, ngram_figures_corpus
#dist = piece.distance(df=ngram_figures_corpus).stack().to_frame() #note that all one column, "0" with the similarity value
#dist = dist.reset_index()
#dist.rename(columns =  {'level_0':'source', 'level_1':'match'}, inplace = True) # doesnt work: , '0':'similarity'
#dist

#old version, which ties into same list end of one movement with beginning of next
#df = harm_corpus['figures'].to_frame()
#ngrams = piece.ngrams(df=df, n=10) #, exclude=['Rest']
#dist = piece.distance(df=ngrams).stack().to_frame()
#dist


#filter automatic similarity detector:
#set_distance_threshold = 3
#distance_factor = set_distance_threshold + 1
#dist.loc[x, "Duration"] > 120
#filtered_dist = dist[dist[0] < distance_factor] # make it 0<dist[0]
#filtered_dist
        
#    = filtered_dist.reset_index()
#filtered_dist.rename(columns =  {'level_0':'source', 'level_1':'match'}, inplace = True) # , '0':'similarity' doesnt work
#filtered_dist
#currently gives distance 3 if one element changes, 838 =>835
#also, changes order of source, but I want source to be ordered
#have source keep its measure, beat numbers 

#For ngram method in creating harm_corpus    
    #ngram_figures = harm['figures'].to_frame() #for ngram similarity method
    #ngram_figures = piece.ngrams(df=ngram_figures, n=quote_length) #for ngram similarity method
    #cleaned_list2.append(ngram_figures) #for ngram similarity method
    #cleaned_list2 = []
#ngram_figures_corpus = pd.concat(cleaned_list2) #for ngram similarity method

#TRASH, I THINK: entire cartesian product, works but only for small number thus useless; must cut duplicates
#df1 = harm_corpus[['Composer','Title','figures_combined']].astype(str)
#df2 = harm_corpus['figures_combined'].astype(str).to_frame()
#df3 = pd.merge(df1.assign(key=1), df2.assign(key=1), on='key').drop('key', axis=1) # GETS RID OF BAR, MEASURE NUMBERS      pd.concat([df1,df2], axis=1)
#df3 #= df3.iloc[0:500]
#df3
#df3['similarity'] = df3[['figures_combined_x', 'figures_combined_y']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)
#df3[df3['similarity'] < 10]


#df2_1 = harm_corpus['figures_combined'].astype(str).to_frame()
#df2_2 = df2_1.reset_index(level=[0,1, 2])
#df2_3 = df2_2 .drop(df2_2.columns[0:3], axis = 1)

#new_df = harm_corpus['figures_combined'].to_frame()
#df1 = new_df.iloc[0:10]
#df2 = df1
#df3 = pd.merge(df1.assign(key=1), df2.assign(key=1), on='key').drop('key', axis=1)
#df3['similarity'] = df3[['figures_combined_x', 'figures_combined_y']].apply(lambda packed_vars : td.levenshtein.distance(*packed_vars), axis=1)
#df3

In [15]:


#DONT TOUCH; LEWIS'S ORIGINAL, WORKS

'''
quote_length = 10

func1 = ImportedPiece.harmonic
list_of_dfs = corpus.batch(func=func1, kwargs={'kind': 'd', 'compound':False}, metadata=False)
func2 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func2, kwargs={'offset':True,'df': list_of_dfs})

cleaned_list = []
for harm in list_of_detail_index: #for each df sitting in the list, arbitrarily called harm, applying these functions (for each df in list, can apply df functions, but not on list as a whole)
    harm = harm.fillna(method="pad")
    harm = harm.replace('Rest', 0)
    harm['figures'] = harm[harm.columns[:-2]].apply( # 0: takes all the columns
        lambda x: ''.join(x.astype(str)), 
        axis=1
    )
    harm = harm.loc[harm['figures'].shift() != harm['figures']]
    for i in range(1, quote_length):
        harm[f"figures{i}"] = harm['figures'].shift(-i) #or "figures"+str(i)
    harm["figures_combined"] = harm[harm.columns[-quote_length:]].apply( 
        lambda x: '_'.join(x.astype(str)),
        axis=1
    )
    harm = harm.drop(harm.columns[-quote_length:-1], axis=1)
    cleaned_list.append(harm)

harm_corpus = pd.concat(cleaned_list)
harm_corpus
'''


'\nquote_length = 10\n\nfunc1 = ImportedPiece.harmonic\nlist_of_dfs = corpus.batch(func=func1, kwargs={\'kind\': \'d\', \'compound\':False}, metadata=False)\nfunc2 = ImportedPiece.detailIndex\nlist_of_detail_index = corpus.batch(func=func2, kwargs={\'offset\':True,\'df\': list_of_dfs})\n\ncleaned_list = []\nfor harm in list_of_detail_index: #for each df sitting in the list, arbitrarily called harm, applying these functions (for each df in list, can apply df functions, but not on list as a whole)\n    harm = harm.fillna(method="pad")\n    harm = harm.replace(\'Rest\', 0)\n    harm[\'figures\'] = harm[harm.columns[:-2]].apply( # 0: takes all the columns\n        lambda x: \'\'.join(x.astype(str)), \n        axis=1\n    )\n    harm = harm.loc[harm[\'figures\'].shift() != harm[\'figures\']]\n    for i in range(1, quote_length):\n        harm[f"figures{i}"] = harm[\'figures\'].shift(-i) #or "figures"+str(i)\n    harm["figures_combined"] = harm[harm.columns[-quote_length:]].apply( \n        